In [486]:
# Importing necessary libraries and importing CSV files into respective dataframes

import pandas as pd

df_CPI = pd.read_csv("/workspaces/MSCI466_Project/data/housing-affordability-in-canada/CPI-inflation-by-region-1914-202.csv")
df_Housing = pd.read_csv("/workspaces/MSCI466_Project/data/housing-affordability-in-canada/housing-supply-price-rental.csv")
df_HPIRegions = pd.read_csv("/workspaces/MSCI466_Project/data/housing-affordability-in-canada/HPI 1981-2022 by regions.csv")
df_HPIFederalandProvincial = pd.read_csv("/workspaces/MSCI466_Project/data/housing-affordability-in-canada/HPI 1981-2022 federal and provincial.csv")
df_Income = pd.read_csv("/workspaces/MSCI466_Project/data/housing-affordability-in-canada/income-distribution-2012-2020.csv")
df_Interest = pd.read_csv("/workspaces/MSCI466_Project/data/housing-affordability-in-canada/Interest and mortgage rates 1951-2022.csv")
df_PopulationDwellings = pd.read_csv("/workspaces/MSCI466_Project/data/housing-affordability-in-canada/population_dwellings_count.csv")
df_PopulationRegion = pd.read_csv("/workspaces/MSCI466_Project/data/housing-affordability-in-canada/population-by-region-1946-2022.csv")
df_Structural = pd.read_csv("/workspaces/MSCI466_Project/data/housing-affordability-in-canada/Structural-dwellings-household-size.csv")
df_CPI['GEO'].unique()


array(['Canada', "St. John's, Newfoundland and Labrador",
       'Halifax, Nova Scotia', 'Saint John, New Brunswick',
       'Québec, Quebec', 'Montréal, Quebec',
       'Ottawa-Gatineau, Ontario part, Ontario/Quebec',
       'Toronto, Ontario', 'Thunder Bay, Ontario', 'Winnipeg, Manitoba',
       'Regina, Saskatchewan', 'Saskatoon, Saskatchewan',
       'Edmonton, Alberta', 'Calgary, Alberta',
       'Vancouver, British Columbia',
       'Charlottetown and Summerside, Prince Edward Island',
       'Newfoundland and Labrador', 'Prince Edward Island', 'Nova Scotia',
       'New Brunswick', 'Quebec', 'Ontario', 'Manitoba', 'Saskatchewan',
       'Alberta', 'British Columbia', 'Whitehorse, Yukon',
       'Yellowknife, Northwest Territories', 'Victoria, British Columbia',
       'Iqaluit, Nunavut'], dtype=object)

In [487]:
# Removes region rows that aren't Canada and selects "All-items"
# df_CPI = df_CPI[df_CPI['GEO'] == 'Canada']
# df_CPI.drop(columns=['GEO'], inplace=True)

provinces = ['Prince Edward Island', 'Nova Scotia', 'New Brunswick', 'Quebec', 'Ontario', 'Manitoba', 'Saskatchewan', 'Alberta']

df_CPI = df_CPI[df_CPI['GEO'].isin(provinces)]


df_CPI = df_CPI[df_CPI['Products and product groups'] == 'All-items']
df_CPI.drop(columns=['Products and product groups'], inplace=True)

# Removes unnecessary columns
df_CPI.drop(columns=['DGUID'], inplace=True)
df_CPI.drop(columns=['UOM'], inplace=True)
df_CPI.drop(columns=['UOM_ID'], inplace=True)
df_CPI.drop(columns=['VECTOR'], inplace=True)
df_CPI.drop(columns=['COORDINATE'], inplace=True)

# Standardizes "Date" name and format
df_CPI['Date'] = df_CPI['REF_DATE'].astype(str) + '01'
df_CPI.drop(columns=['REF_DATE'], inplace=True)

# df_CPI.head(108)

In [488]:
# Standardizes "Date" format and name
df_Housing['Date'] = df_Housing['year'].astype(str).str.replace('.0', '').str.replace('.1', '') + '01'

df_Housing = df_Housing[['Date', 'region', 'population', 'total_dwelling', 'total_dwelling_market', 'labour_participation_rate', 'unemployment_rate', 'disposable_income_change', 
                         'rental_vacancy_rate', 'rental_avilability_rate', 'owned_accommodation_costs_change', 'rental_accommodation_costs_change']]

regions_to_include = ['manitoba', 'prince_edward', 'new_brunswick', 'saskatchewan', 'nova_scotia', 'quebec', 'alberta', 'ontario']

df_Housing = df_Housing[df_Housing['region'].isin(regions_to_include)]

province_mapping = {
    'manitoba': 'Manitoba',
    'prince_edward': 'Prince Edward Island',
    'new_brunswick': 'New Brunswick',
    'saskatchewan': 'Saskatchewan',
    'nova_scotia': 'Nova Scotia',
    'quebec': 'Quebec',
    'alberta': 'Alberta',
    'ontario': 'Ontario'
}

# Update province names in the DataFrame
df_Housing['region'] = df_Housing['region'].map(province_mapping)

df_Housing.shape

(216, 12)

In [489]:

df_HPIRegions = df_HPIRegions[df_HPIRegions['Type'] == 'House and Land']
df_HPIRegions.drop(columns = ['Type'], inplace=True)

# List of columns representing regions
regions_columns = ['Prince Edward Island', 'Nova Scotia', 'New Brunswick', 'Quebec', 'Ontario', 'Manitoba', 'Saskatchewan', 'Alberta']

# Melt the dataframe to transform the columns into a single 'region' column
df_HPIRegions = df_HPIRegions.melt(id_vars=['Month-year'], value_vars=regions_columns, var_name='region', value_name='value')
df_HPIRegions = df_HPIRegions.dropna()
df_HPIRegions.head(20)



# Converts "Month-year" to standard format and renames column to "Date"
df_HPIRegions['Month-year'] = pd.to_datetime(df_HPIRegions['Month-year'], format='%b-%y')
def adjust_year(date):
    
    if date.year - 2000 >= 25 and date.year - 2000 <= 99:
        return date.replace(year=date.year - 100)

    return date
df_HPIRegions['Month-year'] = df_HPIRegions['Month-year'].apply(adjust_year)
df_HPIRegions['Month-year'] = df_HPIRegions['Month-year'].dt.strftime('%Y%m')
df_HPIRegions.rename(columns={'Month-year': 'Date'}, inplace=True)
# df_HPIRegions.rename(columns={'Month-year': 'Date'}, inplace=True)

df_HPIRegions.head(1494)

,Date,region,value
60,198601,Prince Edward Island,83.5
61,198602,Prince Edward Island,83.5
62,198603,Prince Edward Island,83.5
63,198604,Prince Edward Island,83.5
64,198605,Prince Edward Island,83.5
...,...,...,...
1729,200008,Quebec,58.3
1730,200009,Quebec,58.6
1731,200010,Quebec,58.8
1732,200011,Quebec,59.0


In [490]:
df_Housing.shape

(216, 12)

In [491]:
merged_df = pd.merge(df_Housing, df_HPIRegions, left_on=['Date', 'region'], right_on=['Date', 'region'], how='left')
merged_df = pd.merge(merged_df, df_CPI, left_on=['Date', 'region'], right_on=['Date', 'GEO'])
merged_df = merged_df.dropna()
merged_df.drop(columns=['GEO'], inplace=True)
merged_df.rename(columns={'value': 'HPI'}, inplace=True)

merged_df.shape

(216, 14)

In [492]:
merged_df.head(216)

,Date,region,population,total_dwelling,total_dwelling_market,labour_participation_rate,unemployment_rate,disposable_income_change,rental_vacancy_rate,rental_avilability_rate,owned_accommodation_costs_change,rental_accommodation_costs_change,HPI,CPI
0,199001,Manitoba,1105.421,3297,2274,67.300000,7.4,-2.669133,6.1,2.7,4.577354,2.496930,38.3,76.2
1,199101,Manitoba,1109.604,1950,1438,67.025000,8.6,-2.669133,6.2,2.7,5.829327,3.184904,38.8,80.1
2,199201,Manitoba,1112.689,2310,1821,66.475000,9.3,-0.326989,5.8,2.7,0.359644,2.844702,38.3,81.2
3,199301,Manitoba,1117.618,2425,1702,66.850000,9.3,-0.453243,5.6,2.7,-0.028291,1.731113,39.2,83.4
4,199401,Manitoba,1123.230,3197,1664,66.750000,8.8,0.161821,5.5,2.7,1.480992,0.906316,40.7,84.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211,201201,Ontario,13413.702,76742,74414,66.158333,7.9,0.981294,2.5,4.1,1.595535,1.317982,85.5,121.8
212,201301,Ontario,13556.229,61085,58614,66.258333,7.6,2.510899,2.6,4.2,0.523493,1.689593,88.5,123.0
213,201401,Ontario,13685.171,59134,56162,65.841667,7.3,0.572354,2.3,3.8,2.224508,1.338039,89.5,125.9
214,201501,Ontario,13797.038,70156,68292,65.233333,6.8,3.510248,2.4,4.1,2.647799,1.102728,91.2,127.4
